### Notebook to download all the local area development datasets for Bengaluru

NOTE: Downloads the data for all the constituencies, except the last constituency (Yelahanka)

In [ ]:
# imports

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# url of the webpage

BASE_URL = "http://opencity.in/tag/lad-funds/page/{}"

In [ ]:
# method to extract the csv files from individual pages 

def extract_csvs(page_count=2):
    csv_files = []
    for page in range(page_count):
        articles = BeautifulSoup(requests.get(BASE_URL.format(page + 1)).text).find_all('article', {'class': 'mh-posts-list-item'})
        for csv in articles:
            try:
                csv_page = csv.find('a')["href"]
                csv_page_data  = requests.get(csv_page).text
                url_section = str(BeautifulSoup(csv_page_data).find('script', {'type': 'application/ld+json'}))
                csv_url = 'http:/' + ''.join(url_section.split(":")[-7].split('"')[0].split("\\")[2:-1])
                print(csv_url)
            except Exception as e:
                print("Skipping: {}".format(url))
                continue
            if len(csv_url) > 10:
                csv_files.append(csv_url)
    return csv_files

In [ ]:
csv_files = extract_csvs()

In [ ]:
# downloading all the csv files
# make sure the folder 'csvs' exists

for cf in csv_files:
    df = pd.read_csv(cf)
    df.to_csv('./csvs/{}'.format(cf.split('/')[-1]), index=False)